<a href="https://colab.research.google.com/github/hopkinslaurel/SI_data_augmentation/blob/main/Sentinel_2_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount drive to collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import EE and geemap libraries
import ee
import geemap.core as geemap

In [3]:
# Authenticate and initialize EE service
project_id = 'ee-hopkilau'  # Google Cloud project ID

ee.Authenticate()
ee.Initialize(project=project_id)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [4]:
# User defined parameters
cloud_percentage = 20  # filter granules to have low cloud coverage

In [5]:
# GEE code to process Sentinel-2 data https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#description

def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [7]:
1 << 11

2048

In [19]:
# Add raster
jan_2023_climate = (
    ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
    .filterDate('2023-01', '2023-02')
    .first()
)

sentinel_2020_jan = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')  # L1C TOA reflectance
    #TODO: .filterBounds(iterate over regions)
    .filterDate('2020-07-01', '2020-07-31')  # try several dates until we get a clear image -- better to take an average over more images or to have a narrower time window?
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))
    .map(mask_s2_clouds)
)
sentinel_2020_jan

#region = DEFINE
#sentinel_2020_jan_clipped = sentinel_2020_jan.clip(region)

In [18]:
# Instantiate a map object
#m = geemap.Map(center=[30, 0], zoom=2)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(-9.1695, 38.6917, 12)
m.add_layer(sentinel_2020_jan.median(), rgb_vis, 'Sentinel-2 RGB')
m



Map(center=[38.6917, -9.1695], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [ ]:
# Export code
Export.image.toDrive({
    image: clippedImage,
    description: 'Sentinel_2020',
    scale: 10,  # 10m spatial resolution
    region: roi

})




In [30]:
# Define a BBox object.
#bbox = ee.Geometry.BBox(-122.09, 37.42, -122.08, 37.43)
point = ee.Geometry.Point(-122.085, 37.425)


# Apply the buffer method to the BBox object.
#bbox_buffer = bbox.buffer(distance=100)
bbox_buffer = point.buffer(distance=100)  # buffer by distance in m
bbox_bounds = bbox_buffer.bounds()  # get bounds of buffered region
bbox = ee.Geometry.Polygon(bbox_bounds.getInfo()['coordinates'][0])  # extract coordinates for bounding box and turn into a geometry


# Print the result.
display('bbox.buffer(...) =', bbox_buffer)

# Display relevant geometries on the map.
m = geemap.Map()
m.set_center(-122.085, 37.422, 15)
m.add_layer(bbox, {'color': 'black'}, 'Geometry [black]: bbox')
m.add_layer(bbox_buffer, {'color': 'red'}, 'Result [red]: bbox.buffer')
m

'bbox.buffer(...) ='

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.buffer",
    "arguments": {
      "distance": {
        "constantValue": 100
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                -122.085,
                37.425
              ]
            }
          }
        }
      }
    }
  }
})

Map(center=[37.422, -122.085], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [35]:
# Clip image to bounding box
sentinel_2020_jan_clipped = sentinel_2020_jan.median().clip(bbox)


m = geemap.Map()
m.set_center(-122.085, 37.425, 15)
m.add_layer(sentinel_2020_jan_clipped, rgb_vis, 'Sentinel-2 RGB')
m

Map(center=[37.425, -122.085], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [39]:
sentinel_2020_jan.median().projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

In [48]:
# Export image

# How many bands are being extracted here?
image_to_export = sentinel_2020_jan.median().select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12'])  # get band names with image.bandNames()
task = ee.batch.Export.image.toDrive(
    image=image_to_export,
    description='image_export_20m_13bands',  # file name
    folder='ee_demos',
    region=bbox,
    scale=20,  # to match resolution of B11 and B12, per Rouet-Leduc and Hulbert (2024)
    crs='EPSG:4326'  # get projection with image.projection().getInfo()
)
task.start()

In [43]:
sentinel_2020_jan.median().bandNames()
